In [0]:
import pandas as pd
data = pd.read_csv('final.csv')

In [2]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
def getVectors(corpus,vectors,size):
    wordset = set(vectors.wv.index2word) #Checks if the word is in the Word2vec corpus 
    vec = []
    counter = 0
    for words in corpus:    
        featureVec = np.zeros(size,dtype="object")
        for word in words:
            if word in wordset:
                featureVec = np.add(featureVec,vectors[word])
        vec.append(featureVec.T)
        counter = counter + 1
        #print(counter)
    return vec


In [0]:
corpus = []
for words in data.comment:
    words = words.split()
    corpus.append(words)



In [5]:
from keras.utils import to_categorical
Y = to_categorical(data.rating)


Using TensorFlow backend.


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, y_test = train_test_split(corpus,Y, test_size = 0.20, random_state = 123)


In [0]:
import gensim.models.word2vec as wv
vocab_size = 300
min_counts = 10
context = 5
n_workers = 15
down_sample = 1e-2
vectors = wv.Word2Vec(X_train,
                     size = vocab_size,
                     workers =n_workers,
                     window = context,
                     min_count = min_counts,
                     sample = down_sample)



In [8]:
import numpy as np

X_train = getVectors(X_train,vectors,vocab_size)
X_test = getVectors(X_test,vectors,vocab_size)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


In [0]:
X_train = np.array(X_train)
X_test = np.array(X_test)


In [0]:
from keras.models import Sequential
from keras.layers import Dense
from keras import layers
from keras import regularizers

In [11]:
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(output_dim =1024, init = 'uniform', activation = 'relu', input_dim = vocab_size,kernel_regularizer= regularizers.l2(0.0001)))
classifier.add(layers.Dropout(0.2))
# Adding the second hidden layer
classifier.add(Dense(output_dim =1024, init = 'uniform', activation = 'relu'))
#classifier.add(layers.Dropout(0.25))
# Adding the third hidden layer
classifier.add(Dense(output_dim =1024, init = 'uniform', activation = 'relu',kernel_regularizer= regularizers.l2(0.0001)))
#classifier.add(layers.Dropout(0.25))
# Adding the output layer
classifier.add(Dense(output_dim = 2, init = 'uniform', activation = 'softmax'))


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=300, kernel_regularizer=<keras.reg..., units=1024, kernel_initializer="uniform")`
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=1024, kernel_initializer="uniform")`
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", kernel_regularizer=<keras.reg..., units=1024, kernel_initializer="uniform")`
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=2, kernel_initializer="uniform")`
  del sys.path[0]


In [0]:
# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])


In [ ]:
# Fitting the ANN to the Training set
classifier.fit(X_train, Y_train, batch_size = 50, epochs = 60,verbose = False)


In [0]:
y_pred = classifier.predict(X_test)

y_pred = np.argmax(y_pred,axis = 1)
y_test = np.argmax(y_test,axis = 1)


In [15]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix


cm = confusion_matrix(y_test,y_pred)
print(accuracy_score(y_test, y_pred))




0.8829568157368988


In [0]:
classifier.save("deep.h5")